<a href="https://colab.research.google.com/github/imthelizardking/CMP784--Final--Project/blob/main/suggestion_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import and install libraries, dependencies

In [1]:
import tensorflow as tf
import numpy as np
import os
import json

Connect to gdrive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Import data

In [3]:
# import data, from gdrive
dirJson = "/content/gdrive/MyDrive/cmp784_project_literature/colab repo/test_inputs/"
%cd "/content/gdrive/MyDrive/cmp784_project_literature/colab repo/test_inputs"

/content/gdrive/MyDrive/cmp784_project_literature/colab repo/test_inputs


In [16]:
json_files = [pos_json for pos_json in os.listdir(dirJson) if pos_json.endswith('.json')] # get all .json anno. files from dir
NUM_FEATURES = 13 # number of features (types, color, style etc.)
labels_vector = np.zeros((len(json_files),NUM_FEATURES)) # one hot encode matrix
counter_outer = 0 # anno. counter
# dictionary of clothing types, will be expanded
#typeDict ={
#    "vest": 0, #sets index of type to 0, if item exist index0 will be 1
#    "sling dress": 1,
#    "short sleeve dress": 2
#}
typeDict = {
    "short sleeve top": 0, # shifted -1 due to 
    "long sleeve top": 1,
    "short sleeve outwear": 2,
    "long sleeve outwear": 3,
    "vest": 4,
    "sling": 5,
    "shorts": 6,
    "trousers": 7,
    "skirt": 8,
    "short sleeve dress": 9,
    "long sleeve dres": 10,
    "vest dress": 11,
    "sling dress": 12,
}
for json_ in json_files:
  jsonFile = open(json_, 'r')
  values = json.load(jsonFile)
  jsonFile.close()
  counter_inner = 0
  for (k,v) in values.items():
    if k.startswith('item'):
      labels_vector[counter_outer][typeDict[v["category_name"]]] = 1
  counter_outer = counter_outer + 1
print(labels_vector)

[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0.

Trainer:
Import tf keras for lstm model

In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence #gerekmeyebilir
from tensorflow.keras.layers import Input, LSTM, Bidirectional, Dense, Embedding #hepsi gerekmeyebilir

LSTM Model:

In [23]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=13,
        output_dim=13,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)), # making LSTM bidirectional
    tf.keras.layers.Dense(32, activation='relu'), # FC layer for the classification part
    tf.keras.layers.Dense(1) # final FC layer
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 13)          169       
                                                                 
 dense_4 (Dense)             (None, None, 32)          448       
                                                                 
 dense_5 (Dense)             (None, None, 1)           33        
                                                                 
Total params: 650
Trainable params: 650
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def Trainer(x, weights, biases):
  # divide x into two, train and validation

Set Trainer Parameters and Train

In [ ]:
# trainer param.s an go

Test predictor

In [ ]:
# test predictor